In [22]:
import pandas as pd
import itertools as it
import gurobipy as gp
from gurobipy import GRB

In [32]:
def powerset(iterable):
    s = list(iterable)
    return it.chain.from_iterable(it.combinations(s, r) for r in range(1, len(s)+1))

In [144]:
I = list(range(1, 9))
COEFFS = [23, 21, 19, 17, 14, 13, 13, 9]
RHS = 70

perms = list(powerset(I))

ceiling_candidates = []
for p in perms:
    if sum(COEFFS[i-1] for i in p) <= 70:
        ceiling_candidates.append(p)
        
ceilings = []
n = 1
for c in ceiling_candidates:
    is_not_a_subset = True
    for s in ceiling_candidates[n:]: 
        if set(c).issubset(set(s)): 
            is_not_a_subset = False
    if is_not_a_subset: ceilings.append(c)
    n += 1
    
roof_candidates = []
for p in perms:
    if sum(COEFFS[i-1] for i in p) > 70:
        roof_candidates.append(p)
roof_candidates.reverse()
        
roofs = []
n = 1
for c in roof_candidates:
    has_no_subset = True
    for s in roof_candidates[n:]: 
        if set(s).issubset(set(c)): 
            has_no_subset = False
    if has_no_subset: roofs.append(c)
    n += 1

In [180]:
ceilings_book = [(1, 2, 3), (1, 2, 4, 8), (1, 2, 6, 7), (1, 3, 5, 6), (2, 3, 4, 6), (2, 5, 6, 7, 8)]
roofs_book = [(1, 2, 3, 8), (1, 2, 5, 7), (1, 3, 4, 7), (1, 5, 6, 7, 8), (2, 3, 4, 5), (3, 4, 6, 7, 8)]

In [181]:
sum(COEFFS[i-1] for i in (1, 2, 4, 7))

74

In [196]:
model = gp.Model('Optimizing a Constraint')

# add vars
a = model.addVars(I,
                  name='a')
rhs = model.addVar(name='rhs')

# objective function
model.setObjective((rhs - a[3] - a[5]))
#model.setObjective((gp.quicksum(a[i] for i in I)))

# add constraints
model.addConstrs((gp.quicksum(a[i] for i in c) <= rhs for c in ceilings),
                  name='ceiling')

model.addConstrs((gp.quicksum(a[i] for i in r) >= rhs + 1 for r in roofs),
                  name='roof')

model.addConstrs((a[i] >= a[i+1] for i in I[:len(I)-1]))

model.update()

In [197]:
model.write('9 Optimizing a Constraint.lp')
model.optimize()

Gurobi Optimizer version 9.1.0 build v9.1.0rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 93 rows, 9 columns and 471 nonzeros
Model fingerprint: 0x78d58881
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+00]
Presolve time: 0.01s
Presolved: 93 rows, 9 columns, 471 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0   -2.0000000e+30   4.800000e+31   2.000000e+00      0s
      18    2.5000000e+01   0.000000e+00   0.000000e+00      0s

Solved in 18 iterations and 0.02 seconds
Optimal objective  2.500000000e+01


In [268]:
index_sign_x = [(1, ''), (2, '+'), (3, '-'), (4, '+'), (5, '+'), (6, '-'), (7, '+'), (8, '+')]
y = [8, 7, 5, 4, 6, 3 ,1, 2]
x_to_y = dict(zip(index_sign_x, y))

for i, s in index_sign_x:
    print('{}{:.0f}x{}'.format(s, a[x_to_y[(i,s)]].x, i), end=' ')
print('<= {:.0f}'.format(model.ObjVal))